# Как написать своего telegram бота

Во многих проектах в качестве интерфейса используются боты. Давайте научимся их писать, если кто-то не сделал этого без нас.

##  Bot Father

<img src='https://st.tlgrm.ru/aaf/44d/aaf44db7572a2fcafef3ae5ea5c20bb2.png' height='400pt' width='400pt'/>

Чтобы начать делать своего бота, напишите другому боту с ником @BotFather и следуйте его инструкциям. Как только вы создадите бота и получите свой ключ (токен) авторизации, можно будет начать тестировать то, что мы напишем далее. 

# Немного про РосКомНадзор

![rkn](https://tgram.ru/blog/wp-content/uploads/2017/06/telegram-block.png)
Запустить бота на вашем компьютере скорее всего не получится, потому что роскомнадзор заблокировал сервера телеги.

Есть много способов обойти блокировку:
* Использовать <a href='https://rus.windscribe.com/'>какой-нибудь бесплатный VPN</a> (удобно дома).
* Использовать вай-фай с настроенным прокси (например Tinkoff-Guest).
* Работать с зарубежного сервера (heroku точно сработает, Яндекс.Облако неточно)
* Во всех библиотеках есть способ указать, какой использовать прокси: гуглите `proxy list`, выбираете [любой список бесплатных прокси](https://free-proxy-list.net), выбираете оттуда адрес, добавляете его в код (но когда он отвалится, вам будет грустно).
* Чуть более надежный способ: вместо одного адреса вы подкачиваете `proxy list` через [API](https://www.proxy-list.download/api/v1), и если выбранный оттуда адрес падает, просто берете новый.


## pyTelegramBotAPI

Для ботов на питоне существует огромное множество разных библиотек, но мы будем использовать библиотеку pyTelegramBotAPI, так как нам кажется, что она проще других.

In [1]:
!pip install pyTelegramBotAPI

Импортируем библиотеку и начинаем писать функционал

In [2]:
import telebot

Все, что нам нужно, это экземпляр класса TeleBot. В конструктор нужно передать тот самый токен, который вам дал Bot Father.

In [9]:
TOKEN = 'YOUR TOKEN HERE'

bot = telebot.TeleBot(TOKEN)

## Декораторы
![decorator](http://www.xappsoftware.com/wordpress/wp-content/uploads/2017/12/decorators.png)

Обработка сообщений в этой библиотеке реализованна с помощью декораторов. О том, что такое декораторы можно почитать <a href='https://pythonworld.ru/osnovy/dekoratory.html'>тут</a>, но для этого занятия нам не потребуется понимание того, как именно это работает. Главное знать, что декоратор мы используем после знака `@`.

В этой библиотеке два разных декоратора. message_handler и callback_query_handler. Подробнее о втором можете почитать в <a href='https://github.com/eternnoir/pyTelegramBotAPI/blob/master/README.md'>документации</a>, а мы сосредоточимся на первом. Метод, который описан под декоратором message_handler вызывается каждый раз, когда боту приходит сообщение (и еще в паре случаев, о которых тоже написано в <a href='https://github.com/eternnoir/pyTelegramBotAPI/blob/master/README.md'>документации</a>, но это не очень важно, на мой взгляд).

Метод, который мы описываем под декоратором принимает по параметрам message - информацию о сообщении, содержит само сообщение, информацию о юзере, который его отправил, и информацию о чате, в котором отправлено сообщение.

Чтобы отправить ответ пользователю у бота send методы: send_message, send_document и так далее. Все они по параметрам принимают id чата, куда нужно отправить сообщение, в нашем случае это id чата, из которого было получено сообщение (еще можно отправлять всегда юзеру, который сообщение отправил, или вообще рандомному юзеру).

In [3]:
@bot.message_handler()
def get_text_messages(message):
    print('From: ', message.from_user.username)
    print('Message: ', message.text)
    print('Chat type: ', message.chat.type)
    print('---------------------------------')
    bot.send_message(message.chat.id, 'Ты прислал сообщение мне, а я тебе, вот так вот.')

## Запуск бота
Запустить бота не сложно (сложнее его потом убить, но это только в ноутбуках проблема, я просто делаю рестарт кернела, можно попытаться использовать метод bot.stop_polling). Для этого нужно вызвать метод polling. Что делает этот метод: через небольшой промежуток времени спрашивает у серверов telegram, не пришли ли ему новые сообщения и если пришли получает их.

In [5]:
bot.polling()

## О параметрах декоратора

Чтобы не ифать в методе под пустым декоратором, разработчики pyTelegramBotAPI добавили возможность использовать message_handler с параметрами.

Первый параметр - content_types. Метод под декоратором ниже будет вызываться только тогда, когда сообщение в чате с ботом содержит текст или файл:

In [1]:
#добавил сюда импорт и токен еще раз, чтобы вам не приходилось возвращаться к началу ноутбука после рестарта кернела
import telebot

TOKEN = 'YOUR TOKEN HERE'

In [4]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(content_types=['text', 'document'])
def get_text_messages(message):
    bot.send_message(message.chat.id, 'Ты прислал сообщение с текстом или документом, а я тебе текстом ответил.')
    
bot.polling()

Но если вы хотите разделить обработку сообщений с документом и сообщений с текстом, то можно создать два отдельных обработчика:

In [3]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    bot.send_message(message.chat.id, 'Ты прислал сообщение с текстом, а я тебе текстом ответил.')
    
@bot.message_handler(content_types=['document'])
def get_text_messages(message):
    bot.send_message(message.chat.id, 'Ты прислал сообщение с документом, а я тебе текстом ответил.')
    
bot.polling()

Также эта библиотека поддерживает регулярные выражения (мы о них <a href='https://github.com/olpyhn/projects_course/blob/master/parsing/parsing.ipynb'>рассказывали</a>):

In [3]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(regexp='[hH][eE][lL]{2}[oO]')
def get_text_messages(message):
    bot.send_message(message.chat.id, 'Hi, are you from england? Ooooh, i too')
    
bot.polling()

Одним из параметров может быть команда(ы) на которые отзывается этот обработчик:

In [3]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def get_text_messages(message):
    bot.send_message(message.chat.id, 'Стартуем! Ты сказал "Стартуем!"!')
    
bot.polling()

Ну и, наконец, в параметры можно передавать функцию, которая должна решать, обрабатывать это сообщение или нет:

In [4]:
bot = telebot.TeleBot(TOKEN)

ID = #YOUR TELEGRAM ID HERE, можно получить с помощью print(message.from_user.id)
@bot.message_handler(func=lambda message: message.from_user.id == ID)
def handle_text_doc(message):
    bot.send_message(message.chat.id, 'Приветствую, мой Господин!')
    
bot.polling()

## О том, что не нужно писать ботов в ноутбуке

Чтобы не страдать, что нужно перезапускать кернел, лучше писать бота в отдельном файлике. Например, как это сделано <a href='https://github.com/olpyhn/projects_course/blob/master/heroku_bot/main.py'>у меня</a>